In [1]:
import pandas as pd
import numpy as np
import json
import random
import itertools
import ast
from collections import defaultdict
%matplotlib inline

In [2]:
dump_fpath = 'data/declarations.json'

with open(dump_fpath) as f:
    declarations = json.loads(f.read())

In [3]:
random.choice(declarations)

{'bonds': [],
 'incomes': [{'comment': '', 'relative': None, 'size': 2272600.0}],
 'main': {'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'},
  'office': {'id': 698,
   'name': 'Арбитражный суд Ставропольского края',
   'post': 'Заместитель председателя суда',
   'region': {'id': 81, 'name': 'Ставропольский край'},
   'type': {'id': 18, 'name': 'Региональный, без структуры'},
   'url': 'http://www.stavropol.arbitr.ru/'},
  'party': None,
  'person': {'family_name': 'Лысенко',
   'given_name': 'Лариса',
   'id': 27676,
   'name': 'Лысенко Лариса Анатольевна',
   'patronymic_name': 'Анатольевна'},
  'year': 2015},
 'real_estates': [{'comment': '',
   'country': 'Россия',
   'name': 'Квартира',
   'own_type': {'id': 1, 'name': 'В собственности'},
   'region': None,
   'relative': None,
   'share': None,
   'share_type': {'id': 1, 'name': 'Индивидуальная'},
   'square': 56.5,
   'type': {'id': 4, 'name': 'Квартира'}},
  {'comment': '',
   'country': 'Россия',
   'name': '

Total declarations:

In [4]:
len(declarations)

90864

In [5]:
columns = ['person_id', 'person_name', 'year', 
           'office_id', 'office_name', 
           'office_type_id', 'office_type_name',
           'office_region_id', 'office_region_name',
           'income_personal', 'income_relatives', 
           'savings', 'real_estate_amount', 'real_estate_squares']

rows = []
for dec in declarations:
    row = [
        dec['main']['person']['id'],
        dec['main']['person']['name'],
        dec['main']['year'],
        dec['main']['office']['id'],
        dec['main']['office']['name'],
        dec['main']['office']['type']['id'] if dec['main']['office'].get('type') else None,
        dec['main']['office']['type']['name'] if dec['main']['office'].get('type') else None,
        dec['main']['office']['region']['id'] if dec['main']['office'].get('region') else None,
        dec['main']['office']['region']['name'] if dec['main']['office'].get('region') else None,
    ]
    income_personal_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is None])
    income_relatives_sum = sum([inc['size'] for inc in dec['incomes'] if inc['relative'] is not None])
    savings_sum = sum([float(sav.split('руб.')[0].replace(',', '.').replace(' ', '')) for sav in dec['savings']])
    real_estate_amount = len(dec['real_estates'])
    real_estate_squares_sum = sum([(estate['square']  or 0) for estate in dec['real_estates']])
    row += [income_personal_sum, income_relatives_sum, savings_sum, real_estate_amount, real_estate_squares_sum]
    rows.append(row)
len(rows)

90864

In [6]:
df = pd.DataFrame(rows, columns=columns)
df = df.sort_values(by=['person_id', 'year'])
df.head()

,person_id,person_name,year,office_id,office_name,office_type_id,office_type_name,office_region_id,office_region_name,income_personal,income_relatives,savings,real_estate_amount,real_estate_squares
30358,8,Зюганов Геннадий Андреевич,1998,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,124154.00,0.00,0.00,1,150.0
31970,8,Зюганов Геннадий Андреевич,1999,449,Президент Российской Федерации,7,"Федеральный, без региональной структуры",NaN,None,495443.00,10804.00,0.00,3,252.7
127,8,Зюганов Геннадий Андреевич,2006,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,1257784.00,0.00,141740.91,1,167.4
1109,8,Зюганов Геннадий Андреевич,2006,449,Президент Российской Федерации,7,"Федеральный, без региональной структуры",NaN,None,3445291.61,145376.87,400490.32,2,281.3
5520,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,2075542.33,102279.19,0.00,3,448.7


Total unique persons

In [7]:
unique_persons_count = df['person_id'].unique().shape[0]
print(unique_persons_count)

51909


Years

In [8]:
print(df.year.min(), df.year.max())

1998 2017


In [9]:
df['year'].value_counts()

2014    24048
2015    20302
2016    19608
2013    12360
2012     5547
2011     4063
2010     3142
2009     1327
2006      280
2008       86
1998       81
1999       12
2017        8
Name: year, dtype: int64

Remove all declarations from years 1998 - 2009, 2017.

In [10]:
df = df[(df.year >= 2009) & (df.year <= 2016)]
df.head()

,person_id,person_name,year,office_id,office_name,office_type_id,office_type_name,office_region_id,office_region_name,income_personal,income_relatives,savings,real_estate_amount,real_estate_squares
5520,8,Зюганов Геннадий Андреевич,2009,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,2075542.33,102279.19,0.00,3,448.7
0,8,Зюганов Геннадий Андреевич,2010,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,2126487.00,0.00,3126451.00,1,167.4
251,8,Зюганов Геннадий Андреевич,2010,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,2127686.49,168601.94,0.00,3,448.7
1105,8,Зюганов Геннадий Андреевич,2010,449,Президент Российской Федерации,7,"Федеральный, без региональной структуры",NaN,None,7681748.38,349885.97,4343156.26,2,281.3
1213,8,Зюганов Геннадий Андреевич,2011,14,Государственная Дума,7,"Федеральный, без региональной структуры",NaN,None,2177566.64,137656.24,0.00,3,448.7


Declarations count after filtering

In [11]:
df.shape

(90397, 14)

Unique persons after filtering

In [12]:
unique_persons_count = df['person_id'].unique().shape[0]
print(unique_persons_count)

51855


In [13]:
df.to_csv('data/declarations.csv', index=False)

Count number of times persons worked at the same office in the same year

In [14]:
grouped_view = df.groupby(['person_id', 'person_name', 'year']).first()

In [15]:
common_offices = df.groupby(['year', 'office_id'])['person_id'].apply(list).reset_index()
common_offices.head()

,year,office_id,person_id
0,2009,1,"[9133, 9161]"
1,2009,3,"[679, 680, 681, 682, 7298, 7299, 7300, 7301, 7..."
2,2009,4,[5350]
3,2009,5,"[80, 299]"
4,2009,7,"[1882, 19701]"


In [16]:
common_offices.to_csv('data/common_offices.csv', index=False)

## Граф связей

In [17]:
YEARS = range(2009, 2017)

In [18]:
common_offices_filtered = common_offices[common_offices['year'].isin(YEARS)].copy()
common_offices_filtered.head()

,year,office_id,person_id
0,2009,1,"[9133, 9161]"
1,2009,3,"[679, 680, 681, 682, 7298, 7299, 7300, 7301, 7..."
2,2009,4,[5350]
3,2009,5,"[80, 299]"
4,2009,7,"[1882, 19701]"


In [19]:
def populate_dict(person_id_list, graph_dict):
    for pair in itertools.combinations(person_id_list, 2):
        graph_dict[pair] += 1
worked_together_dict = defaultdict(int)    
common_offices_filtered['person_id'].map(lambda person_id_list: populate_dict(person_id_list, worked_together_dict))
len(worked_together_dict)

4591539

In [20]:
# More or equal to the amount of persons
# Less than number of all possible pairs
assert unique_persons_count <= len(worked_together_dict) <= unique_persons_count*(unique_persons_count-1)/2 

In [21]:
with open('data/graph.csv', 'w') as graph_file:
    graph_file.write('person1,person2,times_worked_together\n')
    for pair in sorted(worked_together_dict):
        graph_file.write('{},{},{}\n'.format(pair[0], pair[1], worked_together_dict[pair]))

## Справочник доходов депутатов

In [22]:
df['income'] = df['income_personal'] + df['income_relatives']

In [23]:
income_df = df[(df.year >= 2013) & (df.year <= 2016)].copy()

In [24]:
# Если несколько записей на одного депутата на один год на один офис, то берем первую
officials_year = (income_df
                    .groupby(by=['year', 'person_id', 'person_name', 'office_id'])['income']
                    .first()
                    .to_frame('income')
                    .reset_index())
officials_year.head()

,year,person_id,person_name,office_id,income
0,2013,8,Зюганов Геннадий Андреевич,14,3392882.89
1,2013,9,Алферов Жорес Иванович,14,18133519.25
2,2013,10,Андреев Андрей Анатольевич,14,4368530.71
3,2013,12,Афонин Юрий Вячеславович,14,2802081.80
4,2013,13,Гаврилов Сергей Анатольевич,14,3263364.19


In [25]:
# Суммируем доход в разных офисах
officials_income = (officials_year
                    .groupby(by=['person_id', 'person_name'])['income']
                    .sum()                    
                    .to_frame('income')
                    .reset_index())
officials_income = officials_income.set_index('person_id')
officials_income.head()

,person_name,income
person_id,,
8,Зюганов Геннадий Андреевич,20453218.69
9,Алферов Жорес Иванович,81462661.75
10,Андреев Андрей Анатольевич,14475607.50
12,Афонин Юрий Вячеславович,23774458.69
13,Гаврилов Сергей Анатольевич,19608057.29


In [26]:
officials_income.to_json('data/officials_income.json', orient='index', force_ascii=False, double_precision=2)
officials_income.to_csv('data/officials_income.csv')